# A 'Bet Against Short Sales' Algo
## By Thomas Bueno-Schonig of Alpha Hound Inc
### Objectives
This notebook is a test project, with the intention of building a automated trading system for use with Alpaca's API. The explicit goal is to practice my skills and demonstrate some creativity in sourcing data - not necessarily to build a profitable system. In fact, given the means of sourcing some data input into the algorithm, it will be difficult to backtest and may only be suitable for evalution in walk-forward paper trading. I'm intentionally not using some of the standard Python trading libraries in order to practice my coding skills. 

The algorithm's general strategy is short-term mean reversion on exchange-listed securities that have experienced:
 - Highly negative one-day returns
 - Are subject to high short interest

Our thesis is that given the costs & constraints of short selling, and the natural long bias of markets, stocks with extreme negative returns and short interest are more likely to gain in following trading sessions compared to the overall market. 

### Design Constraints
 - The algorithm will be long-only, therefore exposed systemic risks across the equities market
 - Stocks with insufficient input data will be excluded
 - In the absense of historical short-sale data, the algorithm can only be tested with walk-forward paper trading
 
## Building Our Tradable Universe
### Query Alpaca for All Tradable Securities

In [1]:
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import finviz as fv

In [2]:
#We plug in our API token and establish a connection with our broker, Alpaca
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'
APCA_API_KEY_ID = '###' #API keys scrubbed for Github
APCA_API_SECRET_KEY = '###' #API keys scrubbed for Github
api = tradeapi.REST(APCA_API_KEY_ID,APCA_API_SECRET_KEY,APCA_API_BASE_URL)

In [3]:
#We query Alpaca for all the securities our account has been approved to trade and print out the number
active_assets = api.list_assets(status='active')
print(len(active_assets))

8449


In [4]:
#Our active_assets JSON containts other [mostly useless] data, so we just extract the list of tickers
symbols = [a.symbol for a in active_assets]
len(symbols)

8449

### Fetching Data for Our Analyses & Filtering
We have a list of all the tickers that our Alpaca account is approved to trade - great. But now we need to gather additional data for our analyses, so we can reduce the universe to just the stocks we're looking to trade. We'll be using two primary data sources:
 - Polygon.io, through Alpaca's built-in API calls : for pricing and volume
 - FinViz : For data around each stock's float and short ratios. We'll also use this data to exclude non-stock securities

In [5]:
# Here we make a new dataframe and counter. We loop over each symbol in our tradable universe
# and get all the data Finviz has for that symbol. We convert the resulting dictionary into a dataframe row and
# append it to the dataframe. We print out the counter and statuses with each loop iteration.
fv_df = pd.DataFrame()
c = 0
for symbol in symbols:
    try:
        data = fv.get_stock(symbol)
        data_df = pd.DataFrame(data,index=[0])
        data_df['Symbol'] = symbol
        fv_df = fv_df.append(data_df,ignore_index=True)
        c += 1
        print("Success on ",symbol, "-- ",c," of",len(symbols))
    except:
        c += 1
        print("Fail on",symbol, "-- ",c," of",len(symbols))
        pass

Success on  GOOGL --  1  of 8449
Success on  GOOG --  2  of 8449
Success on  TLTD --  3  of 8449
Success on  FDX --  4  of 8449
Success on  PSC --  5  of 8449
Success on  DIG --  6  of 8449
Success on  MVIN --  7  of 8449
Success on  UEVM --  8  of 8449
Success on  MCY --  9  of 8449
Success on  FMAT --  10  of 8449
Fail on HGLB --  11  of 8449
Success on  PAUG --  12  of 8449
Success on  GULF --  13  of 8449
Success on  HMI --  14  of 8449
Success on  BDCY --  15  of 8449
Fail on KBLM --  16  of 8449
Success on  XSHQ --  17  of 8449
Success on  NXPI --  18  of 8449
Success on  CFG --  19  of 8449
Success on  DAUD --  20  of 8449
Success on  PBI --  21  of 8449
Success on  PRSP --  22  of 8449
Success on  CTT --  23  of 8449
Success on  RYJ --  24  of 8449
Success on  NEXT --  25  of 8449
Success on  CKPT --  26  of 8449
Success on  DDS --  27  of 8449
Success on  CAG --  28  of 8449
Success on  SLF --  29  of 8449
Success on  GLBY --  30  of 8449
Success on  ALNA --  31  of 8449
Succe

Success on  FTXD --  252  of 8449
Success on  DWAW --  253  of 8449
Success on  EVC --  254  of 8449
Success on  VLY --  255  of 8449
Success on  MXF --  256  of 8449
Success on  PNR --  257  of 8449
Success on  NVIV --  258  of 8449
Fail on COF-F --  259  of 8449
Success on  SALT --  260  of 8449
Fail on FNB-E --  261  of 8449
Success on  VG --  262  of 8449
Success on  METX --  263  of 8449
Success on  PHO --  264  of 8449
Success on  ARNC --  265  of 8449
Fail on HWM --  266  of 8449
Success on  AHPI --  267  of 8449
Success on  IDYA --  268  of 8449
Success on  PDM --  269  of 8449
Success on  CFO --  270  of 8449
Success on  NKTR --  271  of 8449
Success on  AWK --  272  of 8449
Fail on HWM- --  273  of 8449
Fail on WUGI --  274  of 8449
Success on  JHMD --  275  of 8449
Success on  WIMI --  276  of 8449
Success on  PTNR --  277  of 8449
Fail on RILYM --  278  of 8449
Success on  FBGX --  279  of 8449
Fail on RCP --  280  of 8449
Fail on MFINL --  281  of 8449
Success on  IVV --  

Fail on SCCB --  500  of 8449
Success on  HEQ --  501  of 8449
Success on  MYRG --  502  of 8449
Success on  SONA --  503  of 8449
Success on  SUSL --  504  of 8449
Success on  BFZ --  505  of 8449
Success on  QADA --  506  of 8449
Success on  UEPS --  507  of 8449
Success on  VNQ --  508  of 8449
Success on  IJJ --  509  of 8449
Success on  SPI --  510  of 8449
Fail on CFB --  511  of 8449
Success on  PMD --  512  of 8449
Success on  AVD --  513  of 8449
Success on  IRBO --  514  of 8449
Success on  HYRE --  515  of 8449
Success on  FDHY --  516  of 8449
Success on  CCU --  517  of 8449
Fail on MS-L --  518  of 8449
Success on  MSBF --  519  of 8449
Success on  HEDJ --  520  of 8449
Fail on NEE-P --  521  of 8449
Success on  BBDC --  522  of 8449
Success on  UAUD --  523  of 8449
Success on  MGLN --  524  of 8449
Success on  PRFT --  525  of 8449
Success on  QLVD --  526  of 8449
Success on  CBTX --  527  of 8449
Success on  AHH --  528  of 8449
Success on  PH --  529  of 8449
Success

Success on  B --  747  of 8449
Success on  HELX --  748  of 8449
Success on  MAYS --  749  of 8449
Success on  DBV --  750  of 8449
Success on  PBEE --  751  of 8449
Success on  ZYME --  752  of 8449
Success on  FIHD --  753  of 8449
Fail on LMST --  754  of 8449
Success on  WCHN --  755  of 8449
Success on  WMB --  756  of 8449
Success on  AFTY --  757  of 8449
Success on  TEX --  758  of 8449
Success on  FISR --  759  of 8449
Success on  CHW --  760  of 8449
Success on  ABTX --  761  of 8449
Success on  LILAK --  762  of 8449
Success on  LEO --  763  of 8449
Success on  RDNT --  764  of 8449
Success on  PBB --  765  of 8449
Fail on BAC-C --  766  of 8449
Success on  UST --  767  of 8449
Fail on FSEA --  768  of 8449
Success on  EDV --  769  of 8449
Success on  GDXJ --  770  of 8449
Success on  CID --  771  of 8449
Success on  AHC --  772  of 8449
Success on  IGOV --  773  of 8449
Success on  HRZN --  774  of 8449
Success on  SPIB --  775  of 8449
Success on  AAXN --  776  of 8449
Suc

Success on  COMT --  995  of 8449
Success on  IHC --  996  of 8449
Success on  CBL --  997  of 8449
Success on  NTAP --  998  of 8449
Fail on SNV-E --  999  of 8449
Success on  WWJD --  1000  of 8449
Success on  SFE --  1001  of 8449
Success on  EOLS --  1002  of 8449
Success on  AKG --  1003  of 8449
Fail on BWB --  1004  of 8449
Fail on BPYU --  1005  of 8449
Success on  ESGN --  1006  of 8449
Success on  HYDB --  1007  of 8449
Success on  OTTW --  1008  of 8449
Success on  SCHA --  1009  of 8449
Success on  ENO --  1010  of 8449
Success on  HDS --  1011  of 8449
Success on  SUMR --  1012  of 8449
Success on  HAL --  1013  of 8449
Success on  IBTX --  1014  of 8449
Fail on NEWTL --  1015  of 8449
Fail on AFGC --  1016  of 8449
Success on  SCHM --  1017  of 8449
Success on  SMIT --  1018  of 8449
Success on  IEF --  1019  of 8449
Success on  WANT --  1020  of 8449
Success on  FKO --  1021  of 8449
Fail on CUBI-C --  1022  of 8449
Fail on FPI-B --  1023  of 8449
Success on  HIFS --  10

...

Success on  BXS --  7470  of 8449
Success on  EVGN --  7471  of 8449
Success on  GRNB --  7472  of 8449
Success on  ASB --  7473  of 8449
Success on  GLUU --  7474  of 8449
Success on  TY --  7475  of 8449
Success on  ABUS --  7476  of 8449
Success on  SMTC --  7477  of 8449
Success on  AIEQ --  7478  of 8449
Success on  LPI --  7479  of 8449
Success on  FORK --  7480  of 8449
Success on  CHE --  7481  of 8449
Success on  PROF --  7482  of 8449
Success on  ACHV --  7483  of 8449
Success on  ARVN --  7484  of 8449
Success on  RTH --  7485  of 8449
Success on  DESP --  7486  of 8449
Success on  RWK --  7487  of 8449
Fail on HNGR --  7488  of 8449
Success on  SPNE --  7489  of 8449
Success on  HCI --  7490  of 8449
Success on  DVA --  7491  of 8449
Success on  DGL --  7492  of 8449
Success on  MYI --  7493  of 8449
Fail on INSU --  7494  of 8449
Success on  LILA --  7495  of 8449
Success on  TECH --  7496  of 8449
Success on  CYRN --  7497  of 8449
Success on  ANGI --  7498  of 8449
Fail 

Success on  PKBK --  7708  of 8449
Success on  FSM --  7709  of 8449
Success on  EHTH --  7710  of 8449
Success on  BST --  7711  of 8449
Success on  BGI --  7712  of 8449
Success on  XPER --  7713  of 8449
Success on  NXJ --  7714  of 8449
Success on  XENE --  7715  of 8449
Success on  XNTK --  7716  of 8449
Success on  MSB --  7717  of 8449
Success on  CRBP --  7718  of 8449
Success on  RLY --  7719  of 8449
Success on  DNP --  7720  of 8449
Success on  ARWR --  7721  of 8449
Success on  CSPI --  7722  of 8449
Success on  SRI --  7723  of 8449
Success on  JBLU --  7724  of 8449
Success on  SUSB --  7725  of 8449
Success on  TIF --  7726  of 8449
Success on  CATS --  7727  of 8449
Success on  GBR --  7728  of 8449
Success on  HTBI --  7729  of 8449
Success on  KBAL --  7730  of 8449
Success on  ZTR --  7731  of 8449
Success on  SHYL --  7732  of 8449
Success on  GCI --  7733  of 8449
Success on  MLND --  7734  of 8449
Success on  LBRDK --  7735  of 8449
Success on  SEM --  7736  of 84

Success on  VRAY --  7946  of 8449
Success on  PRVB --  7947  of 8449
Success on  MQY --  7948  of 8449
Success on  MATX --  7949  of 8449
Success on  AXP --  7950  of 8449
Success on  PSTI --  7951  of 8449
Success on  UTMD --  7952  of 8449
Fail on THBRU --  7953  of 8449
Success on  ADME --  7954  of 8449
Success on  AFIN --  7955  of 8449
Success on  DVP --  7956  of 8449
Success on  NNDM --  7957  of 8449
Success on  ALSK --  7958  of 8449
Success on  BBK --  7959  of 8449
Success on  ATLC --  7960  of 8449
Success on  ZIOP --  7961  of 8449
Success on  TILE --  7962  of 8449
Success on  TENB --  7963  of 8449
Success on  LEG --  7964  of 8449
Fail on AMHC --  7965  of 8449
Success on  RES --  7966  of 8449
Success on  FRPH --  7967  of 8449
Fail on VTIQU --  7968  of 8449
Success on  BLCM --  7969  of 8449
Success on  ALB --  7970  of 8449
Success on  LSTR --  7971  of 8449
Fail on SFTW --  7972  of 8449
Success on  AMCA --  7973  of 8449
Success on  PBF --  7974  of 8449
Success

Success on  ALEX --  8185  of 8449
Success on  MDP --  8186  of 8449
Success on  FISK --  8187  of 8449
Success on  GDNA --  8188  of 8449
Success on  RLI --  8189  of 8449
Success on  AIN --  8190  of 8449
Success on  CAPE --  8191  of 8449
Fail on JFU --  8192  of 8449
Success on  APPN --  8193  of 8449
Fail on RODI --  8194  of 8449
Success on  AINC --  8195  of 8449
Success on  NCV --  8196  of 8449
Success on  MELI --  8197  of 8449
Success on  UCIB --  8198  of 8449
Success on  TNXP --  8199  of 8449
Success on  DAUG --  8200  of 8449
Success on  M --  8201  of 8449
Success on  TFSL --  8202  of 8449
Fail on CRD.A --  8203  of 8449
Success on  AIRR --  8204  of 8449
Success on  IUSB --  8205  of 8449
Success on  PTEN --  8206  of 8449
Success on  STM --  8207  of 8449
Success on  EVK --  8208  of 8449
Success on  CBPO --  8209  of 8449
Success on  CGC --  8210  of 8449
Success on  SPYX --  8211  of 8449
Success on  XDIV --  8212  of 8449
Success on  GRAF --  8213  of 8449
Success

Success on  COLL --  8424  of 8449
Success on  NEE --  8425  of 8449
Success on  EWP --  8426  of 8449
Success on  FIVA --  8427  of 8449
Success on  EWQ --  8428  of 8449
Success on  EFNL --  8429  of 8449
Success on  ECOM --  8430  of 8449
Success on  SFBS --  8431  of 8449
Success on  AGX --  8432  of 8449
Success on  NCR --  8433  of 8449
Success on  CEQP --  8434  of 8449
Success on  SFL --  8435  of 8449
Success on  GSX --  8436  of 8449
Success on  ARDS --  8437  of 8449
Success on  BWG --  8438  of 8449
Success on  USDY --  8439  of 8449
Success on  TCCO --  8440  of 8449
Success on  GLMD --  8441  of 8449
Success on  NIU --  8442  of 8449
Fail on RESD --  8443  of 8449
Success on  PEAK --  8444  of 8449
Success on  CGW --  8445  of 8449
Success on  PLW --  8446  of 8449
Success on  CBB --  8447  of 8449
Success on  IYC --  8448  of 8449
Success on  HCAC --  8449  of 8449


In [6]:
# Now we save the data. Finviz refreshes weekly, and our local memory will probably purge before then. We can re-load 
# the local file in instead of locking up the notebook to pull all the data from Finviz again.
fv_df.to_csv('finviz_extract.csv')

# Here we re-load the Finviz from local CSV - commented out for now
'''
fv_df = pd.read_csv('finviz_extract.csv')
fv_df.set_index('Symbol',inplace=True)
fv_df.head()'''

"\nfv_df = pd.read_csv('finviz_extract.csv')\nfv_df.set_index('Symbol',inplace=True)\nfv_df.head()"

In [7]:
# We create a new df for additional exploration and clean-up while preserving original just incase we need to rollback
df = fv_df
df.set_index('Symbol',inplace=True)
df.head()

,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,...,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
Symbol,,,,,,,,,,,,,,,,,,,,,
GOOGL,S&P 500,24.54,49.18,0.14%,703.78M,9.52%,849.46B,20.76,20.03%,0.00%,...,May 04 AMC,0.00%,2.56M,1207.00,1.80,7.08%,-7.64%,-4.78%,"2,008,221",2.07%
GOOG,S&P 500,24.62,49.16,5.66%,680.92M,9.47%,824.10B,20.54,19.30%,-20.82%,...,May 04 AMC,-,2.35M,1210.28,1.70,7.05%,-7.56%,-4.55%,"1,970,713",2.00%
TLTD,-,-,-,-,-,7.25%,-,-,-,-,...,-,-,112.55K,47.63,-,6.29%,-12.44%,-20.35%,"48,388",0.91%
FDX,S&P 500,-,-0.73,0.10%,279.00M,10.63%,35.03B,11.53,15.08%,2.26%,...,Mar 17 AMC,-,3.18M,125.54,2.40,12.16%,-5.51%,-17.11%,"5,503,654",8.27%
PSC,-,-,-,-,-,14.66%,-,-,-,-,...,-,-,7.38K,22.92,-,8.64%,-15.79%,-24.30%,"12,733",5.43%


In [8]:
# Now we execute another loop to pull down our pricing data over the last two sessions and calculate one-day returns
# NOTE: This loop should be run after the market closes. It currently ignores date and pulls the last daily trading bars.
    # If run during market hours, it'll pull down the current + yesterday's bars for each security prices. 
c = 0
for i in df.index:
    bars = api.get_barset(i,'day',limit=2)
    bars = bars[i]
    prior_period = bars[0].c
    current_period = bars[1].c
    current_volume = bars[1].v
    pct_chg = (current_period - prior_period) / prior_period * 100
    df.at[i,'OneDayChange'] = pct_chg
    df.at[i,'RecentClose'] = prior_period
    df.at[i,'RecentVolume'] = current_volume
    c += 1
    print(c, "of ",len(df.index)," - ",i," - ",pct_chg)

1 of  7506  -  GOOGL  -  2.07360801028347
2 of  7506  -  GOOG  -  2.003354375437205
3 of  7506  -  TLTD  -  1.1228813559321906
4 of  7506  -  FDX  -  8.261469472231814
5 of  7506  -  PSC  -  5.427782888684468
6 of  7506  -  DIG  -  13.612565445026165
7 of  7506  -  MVIN  -  0.4641830127461697
8 of  7506  -  UEVM  -  1.3991176470588333
9 of  7506  -  MCY  -  7.468123861566478
10 of  7506  -  FMAT  -  4.477611940298509
11 of  7506  -  PAUG  -  0.8037225042301088
12 of  7506  -  GULF  -  2.3181233933161924
13 of  7506  -  HMI  -  -0.4796163069544404
14 of  7506  -  BDCY  -  13.898167006109968
15 of  7506  -  XSHQ  -  -0.7734234234234133
16 of  7506  -  NXPI  -  8.955579373585799
17 of  7506  -  CFG  -  5.0691244239631255
18 of  7506  -  DAUD  -  -0.2448022829188764
19 of  7506  -  PBI  -  14.062500000000004
20 of  7506  -  PRSP  -  5.065359477124182
21 of  7506  -  CTT  -  4.238618524332803
22 of  7506  -  RYJ  -  5.442876806607025
23 of  7506  -  NEXT  -  8.376963350785337
24 of  7506  -

193 of  7506  -  XMPT  -  2.31023102310232
194 of  7506  -  EVF  -  3.131524008350738
195 of  7506  -  SHLL  -  0.10101010101009886
196 of  7506  -  SWK  -  6.580909768829233
197 of  7506  -  TRNX  -  2.4390243902439046
198 of  7506  -  PBFX  -  2.559867877786957
199 of  7506  -  INDB  -  4.024144869215283
200 of  7506  -  GRTX  -  -1.79754020813623
201 of  7506  -  IBTE  -  -0.23201856148491384
202 of  7506  -  EPSN  -  6.882591093117406
203 of  7506  -  AEG  -  1.5444015444015458
204 of  7506  -  MACK  -  4.014409221902016
205 of  7506  -  AEF  -  0.375234521575977
206 of  7506  -  IJK  -  4.393605292171995
207 of  7506  -  APTO  -  1.3008130081300824
208 of  7506  -  LUV  -  4.638388770216649
209 of  7506  -  EIDO  -  -1.1620400258231098
210 of  7506  -  GLO  -  2.7857142857142896
211 of  7506  -  FPRX  -  2.9411764705882475
212 of  7506  -  INSI  -  0.7234279354479831
213 of  7506  -  QD  -  -6.133333333333333
214 of  7506  -  CHDN  -  12.327202102136402
215 of  7506  -  CHIK  -  2

sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/day 3 more time(s)...


303 of  7506  -  SDP  -  -9.67122238920237
304 of  7506  -  MMIT  -  2.019192323070768
305 of  7506  -  MUDS  -  -0.0009643201542889773
306 of  7506  -  LUNA  -  3.770491803278696
307 of  7506  -  CHNG  -  2.7559055118110174
308 of  7506  -  LRGF  -  3.6630036630036633
309 of  7506  -  UTSI  -  1.9531250000000049
310 of  7506  -  HNI  -  3.4249471458773884
311 of  7506  -  JKE  -  3.314219580198844
312 of  7506  -  TTGT  -  2.9895366218236243
313 of  7506  -  ARCO  -  4.129793510324474
314 of  7506  -  TOTA  -  -0.0950570342205303
315 of  7506  -  PSEC  -  5.641025641025646
316 of  7506  -  NJAN  -  0.4735128736312628
317 of  7506  -  AE  -  4.918815663801325
318 of  7506  -  FUTY  -  5.124762224460112


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/day 3 more time(s)...


...

7475 of  7506  -  CTAS  -  3.4404902698634428
7476 of  7506  -  ALBO  -  11.186440677966091
7477 of  7506  -  DVD  -  0.9629629629629556
7478 of  7506  -  MSFT  -  0.9908862927396199
7479 of  7506  -  AMZN  -  1.555975343010548
7480 of  7506  -  SSB  -  0.8936883261962332
7481 of  7506  -  MGI  -  4.724409448818902
7482 of  7506  -  COLL  -  3.4600113442994864
7483 of  7506  -  NEE  -  2.1183473389355756
7484 of  7506  -  EWP  -  0.7317073170731638
7485 of  7506  -  FIVA  -  -0.560020494367035
7486 of  7506  -  EWQ  -  1.498287671232883
7487 of  7506  -  EFNL  -  0.6442021803766115
7488 of  7506  -  ECOM  -  6.469002695417796


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/day 3 more time(s)...


7489 of  7506  -  SFBS  -  4.812650395324849
7490 of  7506  -  AGX  -  8.97435897435897
7491 of  7506  -  NCR  -  0.0
7492 of  7506  -  CEQP  -  5.309734513274342
7493 of  7506  -  SFL  -  2.3809523809523876
7494 of  7506  -  GSX  -  -6.165228113440187
7495 of  7506  -  ARDS  -  0.6956521739130441
7496 of  7506  -  BWG  -  1.6697588126159693
7497 of  7506  -  USDY  -  -8.545454545454541
7498 of  7506  -  TCCO  -  -1.4285714285714404
7499 of  7506  -  GLMD  -  3.801169590643272
7500 of  7506  -  NIU  -  0.1447178002894325
7501 of  7506  -  PEAK  -  11.548117154811722
7502 of  7506  -  CGW  -  2.9402940294029527
7503 of  7506  -  PLW  -  -0.7802668009061041
7504 of  7506  -  CBB  -  0.48043925875085985


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/day 3 more time(s)...


7505 of  7506  -  IYC  -  2.5652245255155046
7506 of  7506  -  HCAC  -  0.4960317460317531


#### Sanitizing the Data
Finviz returns numeric data as text strings, which requires regular expressions to convert & evaluate back into numeric data for operations and ranking. Additionally, missing data are valued as hyphens but need to be replaced with NaNs otherwise operations will kick off errors

In [9]:
df = df.replace('-',np.nan)

In [10]:
# Here we make a list of a few columns containing strings we want to convert to numerics
# RegEx is then used to parse strings and replace with mathematical forumulas that can be evaluated in pandas

KMB_cols = ['Shs Outstand', 'Shs Float','Market Cap','Income','Sales']
for col in KMB_cols:
    try:
        df[col] = df[col].replace(r'[KMB]+$', '', regex=True).astype(float) * df[col].str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M','B'], [10**3, 10**6, 10**9]).astype(int)
    except:
        pass
pct_cols = ['Short Ratio', 'Short Float', 'Insider Own','Insider Trans','Inst Own', 'Inst Trans','ROI','ROE','ROA']
for col in pct_cols:
    try:
        df[col] = df[col].replace('%','*.01',regex=True).dropna().map(pd.eval).astype(float)
    except:
        pass

In [11]:
# Resave the joined & cleaned file
df.to_csv('clean_securities_data.csv')
#df.info()

#### Ranking Our Pipeline
Now that we've sourced data for our securities, we need to evaluate them and select which to trade. We'll keep it simple, and rank the universe of securities according to two criteria and then add them together for aggregation. We'll rank by the highest short ratio (highest % of short sales relative to average daily trading volume) and one day draw-down (most negative 1-day return).

In [12]:
df2 = df[df['Shs Float'].notnull()]
cols_to_rank = {'Short Ratio':False,'OneDayChange':True}
df2['AggRank'] = 0
for key,value in cols_to_rank.items():
    name = key + ' Rank'
    df2[name] = df2[key].rank(ascending=value)
    df2['AggRank'] += df2[name]

/home/tom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

Now that we've ranked our universe and selected a the top stocks to trade, I'm curious how they're performing in live trading today. The below cell will re-query for our pricing, and give us a table of the stocks we selected, along with the selection criteria, rankings, and returns data for the current trading day.

In [19]:
# Now that we've ranked our universe, I am curious how these securities are performing in today's trading. So we pull down today's pricing information and append it to the dataframe
pipe = df2[['AggRank','Short Ratio','Short Ratio Rank','OneDayChange','OneDayChange Rank']].sort_values(by='AggRank').head(20)
c = 0
for i in pipe.index:
    bars = api.get_barset(i,'day',limit=2)
    bars = bars[i]
    prior_period = bars[0].c
    prior_volume = bars[0].v
    current_period = bars[1].c
    current_volume = bars[1].v
    current_open = bars[1].o
    current_high = bars[1].h
    current_low = bars[1].l
    oc_pct = (current_period - current_open) / current_open *100
    pct_chg = (current_period - prior_period) / prior_period * 100
    pipe.at[i,'TodayChange'] = pct_chg
    pipe.at[i,'YesterdayClose'] = prior_period
    pipe.at[i,'TodayVolume'] = current_volume
    pipe.at[i,'TodaySpot'] = current_period
    pipe.at[i,'TodayOpen'] = current_open
    pipe.at[i,'TodayHigh'] = current_high
    pipe.at[i,'TodayLow'] = current_low
    pipe.at[i,'OpenClose%Chg'] = oc_pct
    c += 1
    print(c, "of ",len(pipe.index)," - ",i," - ",pct_chg)
pipe

1 of  20  -  CIA  -  3.0575539568345476
2 of  20  -  SQBG  -  2.942541436464087
3 of  20  -  HKIB  -  2.32558139534885
4 of  20  -  OPTN  -  5.2053333333333285
5 of  20  -  TLGT  -  -3.8461538461538494
6 of  20  -  INOV  -  3.2484076433121123
7 of  20  -  CVIA  -  13.839959738298957
8 of  20  -  WMK  -  1.9526763151849331
9 of  20  -  PEI  -  25.885398282150852
10 of  20  -  DUO  -  7.425742574257425
11 of  20  -  CTRC  -  5.06999176567463
12 of  20  -  NXE  -  1.8372703412073448
13 of  20  -  DDS  -  6.4769975786924965
14 of  20  -  UONEK  -  14.450261780104704
15 of  20  -  CBL  -  14.310123878684324
16 of  20  -  TBIO  -  2.6289180990899874
17 of  20  -  TRHC  -  7.224651259629396
18 of  20  -  WBAI  -  2.1176470588235263
19 of  20  -  NCNA  -  6.679035250463829
20 of  20  -  KOPN  -  -2.7292549871521627


,AggRank,Short Ratio,Short Ratio Rank,OneDayChange,OneDayChange Rank,TodayChange,YesterdayClose,TodayVolume,TodaySpot,TodayOpen,TodayHigh,TodayLow,OpenClose%Chg
Symbol,,,,,,,,,,,,,
CIA,45.0,66.75,3.0,-7.794362,42.0,3.057554,5.560000,3296.0,5.730000,5.7600,5.8200,5.690000,-0.520833
SQBG,73.0,29.73,18.0,-7.179487,55.0,2.942541,0.181000,570.0,0.186326,0.1888,0.1888,0.186326,-1.310381
HKIB,115.0,45.96,7.0,-4.867257,108.0,2.325581,8.600000,11202.0,8.800000,8.8000,8.8000,8.790000,0.000000
OPTN,119.0,28.40,22.0,-5.063291,97.0,5.205333,3.750000,29128.0,3.945200,3.8600,3.9900,3.820000,2.207254
TLGT,130.0,15.14,124.0,-18.750000,6.0,-3.846154,0.260000,105199.0,0.250000,0.2688,0.2800,0.240500,-6.994048
INOV,151.0,20.12,59.0,-5.250453,92.0,3.248408,15.700000,46124.0,16.210000,16.0100,16.2500,15.665000,1.249219
CVIA,154.5,18.13,81.5,-6.052009,73.0,13.839960,0.397400,216688.0,0.452400,0.4350,0.4603,0.410000,4.000000
WMK,192.0,18.96,70.0,-4.623138,122.0,1.952676,43.530000,4947.0,44.380000,43.7500,44.6100,43.580000,1.440000
PEI,215.5,15.04,125.5,-5.261398,90.0,25.885398,0.849900,2997700.0,1.069900,0.9600,1.0800,0.850000,11.447917


Not bad, since almost all of our stocks are up more than the overall market - in some cases, considerably. However, the gains during regular trading hours ('OpenClose%Chg' column) is considerably lower. This illustrates the difficulty in actually capturing alpha: even where it can be identified, trading in a way that allows you to capture it is a different, harder task.

Next up will be coding the account and order management functions, to put our stock selection to use. Below are noted some thoughts on how / where to improve this notebook whenever I revisit it.

##### Areas for Improvement
 - Add timing function to schedule execution of:
     - Finviz query (weekly on Monday nights)
     - Pricing query (on trading days, after market close)
 - Replace the 'OneDayReturn' factor with something more robust (eg. most negative short term moving averages)
 - Add factor for liquidity:
     - Filter universe to remove securities too illiquid to trade algorithmically
     - Add / test factor to identify and prioritize stocks falling on low liquidity 
     